In [ ]:
## sell players; returns trade_id, which can be used to check for trade status

#check status of trade
core.tradeStatus(trade_id=247121399442)
#core.tradeStatus(trade_id=247121787886)

In [ ]:
####### old functions #############

def get_players_to_buy_df(df, score=0.8):
    
    time_start = datetime.now()
    print(time_start)
    
    df['Current'] = df['Current'].astype(int)
    df['14d Median'] = df['14d Median'].astype(int)

        #buy rule: current 20% < 14d median
    try:
        df['Score'] = (df['Current']/df['14d Median']) #lower the better
    except ZeroDivisionError:
        df['Score'] = 1


    df['Buy?'] = np.where(df['Score'] < score, 1, 0)

    df1 = df[df['Buy?']==1].sort_values('Score')
    df1.drop_duplicates(inplace=True)

    print(str(df1.shape[0]) + " potential buys")
    print("Current rule to buy: <" + str(score*100) + "% of 14-day Median")
    print("Estimated cost to execute (assuming 3 each): " + str(sum(df1['Current']) * 3))

        #create a watchlist of sorts? those with current price < 85 or 90% of 14d median? 
        #(prioritize searching thru this watchlist rather than all)
    
    time_end = datetime.now()
    time_delta = time_end - time_start
    
    return df1


def get_player_price_stats(resource_id_df): #15 mil; max value for FUTBIN   

    if type(resource_id_df) == int:
        resource_id_df = pd.DataFrame(columns=['resourceId']) 
    
    #add rating
    player_prices_df = pd.DataFrame(columns =['resourceId', 
                                              #'Current', 'Time', 
                                              'Date Added',
                                        'YTD Mean', 'YTD Median', 'YTD Max', 'YTD Min', 
                                        '30d Mean', '30d Median', '30d Max', '30d Min',
                                       '14d Mean', '14d Median', '7d Mean', '7d Median',
                                       'Yest'])

    for resource_id in resource_id_dict: 
            #hist price data
        hist_price_df = pd.DataFrame(columns=['Date', 'Price'])

        hist_price_url = 'https://www.futbin.com/19/playerGraph?type=daily_graph&year=19&player={0}'
        hist_price_data = requests.get(hist_price_url.format(resource_id)).json()

            # sleep for 5-10 seconds
        time.sleep(random.randint(1,3))

        for price in hist_price_data['ps']:
            #There is extra zeroes in response.
            date = datetime.utcfromtimestamp(price[0] / 1000).strftime('%Y-%m-%d')
            price = price[1]

            hist_price_df = hist_price_df.append({'Date': date, 
                                            'Price': price}, ignore_index=True)

        mean_ytd = int(round(hist_price_df['Price'].mean()))
        median_ytd = int(round(hist_price_df['Price'].median()))
        max_ytd = hist_price_df['Price'].max()
        min_ytd = hist_price_df['Price'].min()

        mean_30d = int(round(hist_price_df['Price'].tail(30).mean()))
        median_30d = int(round(hist_price_df['Price'].tail(30).median()))
        max_30d = hist_price_df['Price'].tail(30).max()
        min_30d = hist_price_df['Price'].tail(30).min()

        mean_14d = int(round(hist_price_df['Price'].tail(14).mean()))
        median_14d = int(round(hist_price_df['Price'].tail(14).median()))

        mean_7d = int(round(hist_price_df['Price'].tail(7).mean()))
        median_7d = int(round(hist_price_df['Price'].tail(7).median()))

        price_yest = hist_price_df['Price'].tail(1).iloc[0]

        date_added = hist_price_df['Date'].head(1).iloc[0]

        #append to dataframe
        player_prices_df_single = pd.DataFrame(columns =['resourceId','Name', 
                                                         #'Current', 'Time', 
                                                          'Date Added',
                                        'YTD Mean', 'YTD Median', 'YTD Max', 'YTD Min', 
                                        '30d Mean', '30d Median', '30d Max', '30d Min',
                                       '14d Mean', '14d Median', '7d Mean', '7d Median',
                                       'Yest'])

        #name = fut.core.get_player_name(int(resource_id))

        player_prices_df_single = player_prices_df_single.append({'resourceId': resource_id,
                                                                  #'Name': name,
                                            #'Current': current_price,
                                            #'Time': current_price_time,
                                              'Date Added': date_added,
                                              'YTD Mean': mean_ytd, 
                                              'YTD Median': median_ytd, 
                                              'YTD Max': max_ytd, 
                                              'YTD Min': min_ytd, 
                                              '30d Mean': mean_30d, 
                                              '30d Median': median_30d, 
                                              '30d Max': max_30d, 
                                              '30d Min': min_30d,
                                              '14d Mean': mean_14d, 
                                              '14d Median': median_14d, 
                                              '7d Mean': mean_7d, 
                                              '7d Median': median_7d,
                                               'Yest': price_yest
                                             }, ignore_index=True) 



        #if buy = 1, then display
        #display(player_prices_df_single)

        player_prices_df = player_prices_df.append(player_prices_df_single, ignore_index=True)
    #else:
    #    continue
        #print(name, rating, position, version, asset_id, resource_id + " is not added")

    #time.sleep(random.randint(10,15))
    print("TODO: fix YTD Min = 0 for players; maybe because added late? or not available on market?")
    return player_prices_df


def get_player_name(resource_id):
    
    players_db = fut.core.players()
    
    asset_id = players_db.baseId(resource_id) # baseId() takes in type 'int'
    first_last_name = players_db[asset_id].get('firstname') + ' ' + players_db[asset_id].get('lastname')
    surname = players_db[asset_id].get('surname')
    
    if surname is None:
        name = first_last_name
    elif len(surname.split()) < len(first_last_name.split()):
        name = surname
    else:
        name = first_last_name
    
    return name


def get_hist_prices_futbin(df, db):

    yesterday = (datetime.utcnow() - timedelta(1)).strftime('%Y-%m-%d')
    
    df['resourceId'] = df['resourceId'].astype(int) # change to 'int' so that baseId can take in
    
    if
    
    for index, row in df.iterrows(): 
            # resource_id
        resource_id_to_add = int(row['resourceId']) # convert to int so that baseId() can take in
        print(resource_id_to_add)
        #name = get_player_name(resource_id)  
        
        indiv_player_prices = hist_price_db[hist_price_db['resourceId']==resource_id_to_add]
        
        if yesterday in indiv_player_prices['Date'].unique():
            pass
            print("already exists")
        else:
                #hist price data 
            hist_price_url = 'https://www.futbin.com/19/playerGraph?type=daily_graph&year=19&player={0}'
            hist_price_data = requests.get(hist_price_url.format(resource_id_to_add)).json()

                # sleep for 5-10 seconds
            time.sleep(random.randint(1,3))

            for price in hist_price_data['ps']:
                #There is extra zeroes in response.
                date = datetime.utcfromtimestamp(price[0] / 1000).strftime('%Y-%m-%d')
                price = price[1]

                hist_price_db = hist_price_db.append({'resourceId': resource_id_to_add,
                                                      'Date': date, 
                                                'Price': price}, ignore_index=True)
    
        hist_price_db.drop_duplicates(inplace=True)
    
    #hist_price_db.reset_index(inplace=True)
    hist_price_db.to_sql(name='player_prices', index=False, if_exists='append', con=conn)
    
    return hist_price_db

def market_scan(min_rating=75, max_rating=99, min_price=200, max_price=999999999):#, version='gold_rare'):

    time_start = datetime.now()
    print(time_start)
        # !method to get resource_ids; returns a df 
    shortlisted_df = futbin_html_scrape(min_rating = min_rating, 
                                        max_rating = max_rating, 
                                        min_price= min_price,
                                        max_price = max_price
                                       #,version = version
                                       ) 
        # !method
    player_prices_df = get_futbin_hist_price(shortlisted_dict) ## !method
        # does not get player price stats!

        #merge dataframes
    shortlisted_df1 = shortlisted_df.merge(player_prices_df, how='left', on='resourceId')

    time_end = datetime.now()
    time_delta = time_end - time_start

    #print(players_db_df.shape[0] + " rows in " + time_delta + " seconds")
    print(time_end)
    print(time_delta)
    print('TODO: Keep URL and print it')

    return shortlisted_df1

def get_current_price(df): 
    # take in both a single resource_id and a dataframe
    
        # convert to dataframe if its a single resource_id
    if type(df) == int:
        df = pd.DataFrame({'resourceId': [df]})
    
        # drop columns to be refreshed
    if 'Current' in df.columns:
        df = df.drop(['Current'], axis='columns')

    if 'Name_x' in df.columns: 
        df =df.drop(['Name_x'], axis='columns')
        
    if 'Name_y' in df.columns: 
        df = df.drop(['Name_y'], axis='columns')
        
    if 'Score' in df.columns: 
        df = df.drop(['Score'], axis='columns')
        
    if 'Buy?' in df.columns: 
        df = df.drop(['Buy?'], axis='columns')
        
    df['resourceId'] = df['resourceId'].astype(int) # change to 'int' so that baseId can take in
    
        # create df to populate with refreshed prices
    current_price_df = pd.DataFrame(columns=['resourceId', 'Name', 'Current', 'Time'])
    
    resource_ids = df['resourceId'].unique()
    
        # iterate through 
    for resource_id in resource_ids:           
            # convert to int so that baseId() can take in
        name = get_player_name(resource_id)
        
            # current price data
        url = requests.get('https://www.futbin.com/19/playerPrices?player={0}'.format(resource_id))
        current_price_data = url.json()
            
            # sleep
        time.sleep(random.randint(1,3))
        
        current_price = int(current_price_data[str(resource_id)]['prices']['ps']['LCPrice'].replace(',', ''))
        current_price_time = current_price_data[str(resource_id)]['prices']['ps']['updated']
            
            # append to df
        current_price_df = current_price_df.append({'resourceId': resource_id, 
                                                  'Name': name, 
                                                  'Current': current_price, 
                                                 'Time': current_price_time}, ignore_index=True)
    
    refreshed_prices_df = df.merge(current_price_df, on='resourceId', how='left') 
    
    print("TODO: do a unique to reduce # of requests to futbin")
    
    return refreshed_prices_df

def futbin_html_scrape(min_rating=75, max_rating=99, min_price=200, max_price=99999999999): #, version='gold_rare'):
    # method goes to player page with 30 players
    # remove this gold_rare later

        # this checks if version is specified; i.e. to remove non-gold cards to reduce # of players to be searched
    '''
    if version != None:
        version = '&version=' + version
    if min_rating != None:
        min_rating = '&player_rating=' + str(min_rating) #this line is useless atm
    '''

        # this chunk provides an estimate of # of players to be searched and the est. time taken
    first_page_url = 'https://www.futbin.com/19/players?page=1&player_rating={0}-{1}&ps_price={2}-{3}&sort=ps_price&order=desc'
    first_page_url_format = first_page_url.format(min_rating, max_rating, min_price, max_price)#, version)
    first_page = requests.get(first_page_url_format)
    time.sleep(random.randint(1,3))
    first_page_soup = BeautifulSoup(first_page.text, 'lxml')
    if first_page_soup.find_all('li', class_='page-item ') == []:
        num_pages = 1
    else:
        num_pages = int(first_page_soup.find_all('li', class_='page-item ')[-1].find('a').find(text=True))
    print(str(num_pages) + " pages/requests: " + str(num_pages * 30) + " players expected")
    print("Expected time to complete: " + str(num_pages*30 / 6) + "mins")

        #
    A=[] #resource_id
    B=[] #name
    C=[] #rating
    D=[] #current price

    for i in range(num_pages):
        url = 'https://www.futbin.com/19/players?page={0}&player_rating={1}-{2}&ps_price={3}-{4}&sort=ps_price&order=desc'
            #i+1 to start from page 1 rather than 0

        url_format = url.format(i+1, min_rating, max_rating, min_price, max_price)#, version) 
        page = requests.get(url_format)
            #sleep
        time.sleep(random.randint(1,3))
        soup = BeautifulSoup(page.text, 'lxml')
        table = soup.find('table', class_='table table-bordered table-hover table-responsive w-100 d-block d-md-table')

        for row in table.find_all('tr'):
            player_bio = row.find('td', class_='table-row-text row') 

            #find url for actual player page
            player_page_url_container = player_bio.find('div', class_='d-inline pt-2 pl-3')
            player_page_url = player_page_url_container.find('a', class_='player_name_players_table').get('href')
            player_page_url = 'https://www.futbin.com' + str(player_page_url).replace(" ", "%20")

            #find player name
            name = player_page_url_container.find_all('div')[0].find('a', class_='player_name_players_table').find(text=True)

            #find player rating
            rating = row.find('span', class_='form rating ut19  gold rare')
            #position = 

                ####### going to actual player/next page ########
            player_page = requests.get(player_page_url)
                #sleep
            time.sleep(random.randint(1,3))
            player_page_soup = BeautifulSoup(player_page.text, 'lxml')
            resource_id_container = player_page_soup.find('div', class_='site-player-page')
            resource_id_container_div = resource_id_container.find('div', class_='container')
            resource_id = resource_id_container_div.find('div').get('data-player-resource')

            #get current price from previous page; might wanna get it from actual player page, or use the get_current_price
            #method by passing it the resource_id found (which will create more requests but might be necessarily to get
            # median current price - prevent issues)
            current_price = str(row.find_all('td')[4].find(text=True)) #convert to string

            if current_price.find('K') != -1:
                current_price = float(current_price.replace('K', '')) * 1000 #change to int?
            elif current_price.find('M') != -1:
                current_price = float(current_price.replace('M', '')) * 1000000
            else:
                current_price = float(current_price)

            current_price = int(current_price)

            A.append(resource_id)
            B.append(name)
            C.append(rating)
            D.append(current_price)

        df = pd.DataFrame()
        df['resourceId'] = A
        df['Name'] = B
        df['Rating'] = C
        df['Current'] = D
        #end of futbin html data
    print("TODO: fix rating and name_y?")

def get_tradepile_prices():  # profit_threshold is in percentage
        ## get prices of player cards in tradepile

    time_start = datetime.now()
    print(time_start)

    tradepile_df = get_tradepile_players() ## !!! method 

    #tradepile_ids = tradepile_df['resourceId'].unique()

    current_price_df_final = pd.DataFrame(columns=['resourceId', 'Current', 'Profit Threshold'])
    
    current_price_df = get_current_price(tradepile_df)
         #sleep
    time.sleep(random.randint(1,5))
    
    for index, row in current_price_df.iterrows():
        resource_id = row['resourceId']
        current_price = row['Current']

        price_step, profit_threshold = get_price_profit_steps(current_price)
        current_price_df_final = current_price_df_final.append({'Current': current_price,
                                                   'Profit Threshold': profit_threshold
                                                   },ignore_index=True)

    tradepile_df = tradepile_df.merge(current_price_df_final, on='resourceId', how='left')  

    #calculate profit
    tradepile_df['Profit'] = tradepile_df['Current'] - tradepile_df['Bought']

    tradepile_df['Profit %'] = np.where(tradepile_df['Bought']==0, 0,\
                                        tradepile_df['Profit']/tradepile_df['Bought']*100).astype(int)
    
    
    
    tradepile_df['Sell?'] = np.where(tradepile_df['Profit %'] >= tradepile_df['Profit Threshold'], 1, 0)

    tradepile_df.sort_values('Profit %', ascending=False, inplace=True)

    time_end = datetime.now()
    time_delta = time_end - time_start
    print(time_end)
    print(time_delta)

    return tradepile_df





def get_hist_prices(resource_id_dict, max_price=15000000): #15 mil; max value for FUTBIN   

    if type(resource_id_dict) == int:
        resource_id_dict = {resource_id_dict} 
    
    
    
    #add rating
    player_prices_df = pd.DataFrame(columns =['resourceId', 
                                              #'Current', 'Time', 
                                              'Date Added',
                                        'YTD Mean', 'YTD Median', 'YTD Max', 'YTD Min', 
                                        '30d Mean', '30d Median', '30d Max', '30d Min',
                                       '14d Mean', '14d Median', '7d Mean', '7d Median',
                                       'Yest'])

    for resource_id in resource_id_dict: 
            #hist price data
        hist_price_df = pd.DataFrame(columns=['Date', 'Price'])

        hist_price_url = 'https://www.futbin.com/19/playerGraph?type=daily_graph&year=19&player={0}'
        hist_price_data = requests.get(hist_price_url.format(resource_id)).json()

            # sleep for 5-10 seconds
        time.sleep(random.randint(1,3))

        for price in hist_price_data['ps']:
            #There is extra zeroes in response.
            date = datetime.utcfromtimestamp(price[0] / 1000).strftime('%Y-%m-%d')
            price = price[1]

            hist_price_df = hist_price_df.append({'Date': date, 
                                            'Price': price}, ignore_index=True)

        mean_ytd = int(round(hist_price_df['Price'].mean()))
        median_ytd = int(round(hist_price_df['Price'].median()))
        max_ytd = hist_price_df['Price'].max()
        min_ytd = hist_price_df['Price'].min()

        mean_30d = int(round(hist_price_df['Price'].tail(30).mean()))
        median_30d = int(round(hist_price_df['Price'].tail(30).median()))
        max_30d = hist_price_df['Price'].tail(30).max()
        min_30d = hist_price_df['Price'].tail(30).min()

        mean_14d = int(round(hist_price_df['Price'].tail(14).mean()))
        median_14d = int(round(hist_price_df['Price'].tail(14).median()))

        mean_7d = int(round(hist_price_df['Price'].tail(7).mean()))
        median_7d = int(round(hist_price_df['Price'].tail(7).median()))

        price_yest = hist_price_df['Price'].tail(1).iloc[0]

        date_added = hist_price_df['Date'].head(1).iloc[0]

        #append to dataframe
        player_prices_df_single = pd.DataFrame(columns =['resourceId','Name', 
                                                         #'Current', 'Time', 
                                                          'Date Added',
                                        'YTD Mean', 'YTD Median', 'YTD Max', 'YTD Min', 
                                        '30d Mean', '30d Median', '30d Max', '30d Min',
                                       '14d Mean', '14d Median', '7d Mean', '7d Median',
                                       'Yest'])

        #name = get_player_name(int(resource_id))

        player_prices_df_single = player_prices_df_single.append({'resourceId': resource_id,
                                                                  #'Name': name,
                                            #'Current': current_price,
                                            #'Time': current_price_time,
                                              'Date Added': date_added,
                                              'YTD Mean': mean_ytd, 
                                              'YTD Median': median_ytd, 
                                              'YTD Max': max_ytd, 
                                              'YTD Min': min_ytd, 
                                              '30d Mean': mean_30d, 
                                              '30d Median': median_30d, 
                                              '30d Max': max_30d, 
                                              '30d Min': min_30d,
                                              '14d Mean': mean_14d, 
                                              '14d Median': median_14d, 
                                              '7d Mean': mean_7d, 
                                              '7d Median': median_7d,
                                               'Yest': price_yest
                                             }, ignore_index=True) 



        #if buy = 1, then display
        #display(player_prices_df_single)

        player_prices_df = player_prices_df.append(player_prices_df_single, ignore_index=True)
    #else:
    #    continue
        #print(name, rating, position, version, asset_id, resource_id + " is not added")

    #time.sleep(random.randint(10,15))
    print("TODO: fix YTD Min = 0 for players; maybe because added late? or not available on market?")
    return player_prices_df


def check_trade_status():
    ## get status of on-going trades
    ## issues with the IDs

    tradepile = core.tradepile()

    tradelist_df = pd.DataFrame(columns=['itemId', 'assetId', 'resourceId', 'Name', 'Bought', 'PRP (Max)', 
                                         'Current Bid', 'BIN', 'Expires'])

    for card in tradepile:
        if card.get('tradeState')=='active':
        #other state is 'expired'


            item_id = card.get('id')
            asset_id = card.get('assetId')

            name = players_db[asset_id].get('firstname') + ' ' + players_db[asset_id].get('lastname')

            resource_id = card.get('resourceId')
            last_sale_price = card.get('lastSalePrice')
            prp_max = card.get('marketDataMaxPrice')
            trade_id = card.get('tradeId')
            timestamp = card.get('timestamp')
            bin_price = card.get('buyNowPrice')
            current_bid = card.get('currentBid')
            expires = card.get('expires')
            #how about sold stuff

            tradelist_df = tradelist_df.append({'itemId': item_id, 
                                              'resourceId': resource_id,
                                                'Name': name,
                                              'Bought': last_sale_price,
                                              'PRP (Max)': prp_max,
                                                'Current Bid' : current_bid,
                                                'BIN' : bin_price,
                                                'Expires': expires
                                               }, ignore_index=True)

    return tradelist_df

def update_sell_percentage(df, profit_threshold=50):
    #change sell %tage

    df['Sell?'] = np.where(df['Profit %'] >= profit_threshold, 1, 0)

    return df


def get_tradepile_prices_old(profit_threshold=50):  # profit_threshold is in percentage
        ## get prices of player cards in tradepile

    time_start = datetime.now()
    print(time_start)

    tradepile = core.tradepile()
    time.sleep(random.randint(1,3))

    #tradepile_df = pd.DataFrame(columns=['itemId', 'resourceId', 'Name', 'Current', 'Time'])
    tradepile_df = pd.DataFrame(columns=['itemId', 'resourceId', 'Bought'])#, 'PRP (Max)', 'PRP (Min)'])

    for card in tradepile:
        if card.get('tradeState') != 'active'\
        and card.get('itemType') == 'player'\
        and card.get('rareflag')!= 0\
        and card.get('tradeState') != 'closed':

            item_id = card.get('id')
            #asset_id = card.get('assetId')
            resource_id = card.get('resourceId')
            last_sale_price = card.get('lastSalePrice')
            #prp_max = card.get('marketDataMaxPrice')
            #prp_min = card.get('marketDataMinPrice')

            #temp_tradepile_df = pd.DataFrame(columns=['itemId', 'resourceId', 'Bought', 'Profit', 'PRP (Max)', 'PRP (Min)'])

            tradepile_df = tradepile_df.append({'itemId': item_id, 
                                                  'resourceId': resource_id, 
                                                  'Bought': last_sale_price
                                                  #'PRP (Max)': prp_max,
                                                  #'PRP (Min)': prp_min
                                                },
                                               ignore_index=True)
            #sleep
        time.sleep(random.randint(1,5))

    tradepile_ids = tradepile_df['resourceId'].unique()

    current_price_df = pd.DataFrame()

    for resource_id in tradepile_ids:
        current_price = get_current_price(resource_id) ## !!! method 
            #sleep
        time.sleep(random.randint(1,5))
        current_price_df = current_price_df.append(current_price)

    tradepile_df = tradepile_df.merge(current_price_df, on='resourceId', how='left')  

    #calculate profit
    tradepile_df['Profit'] = tradepile_df['Current'] - tradepile_df['Bought']

    tradepile_df['Profit %'] = np.where(tradepile_df['Bought']==0, 0,\
                                        tradepile_df['Profit']/tradepile_df['Bought']*100).astype(int)

    tradepile_df['Sell?'] = np.where(tradepile_df['Profit %'] >= profit_threshold, 1, 0)

    tradepile_df.sort_values('Profit %', ascending=False, inplace=True)

    time_end = datetime.now()
    time_delta = time_end - time_start
    print(time_end)
    print(time_delta)

    return tradepile_df

def get_price_graphs():
    curr_price = pd.DataFrame(columns=['Name', 'Current', 'Time'])

    for (name,id) in player_ids.items():

        curr = requests.get('https://www.futbin.com/19/playerPrices?player={0}'.format(id))
        curr_data = curr.json()
        current_price = curr_data[str(id)]['prices']['ps']['LCPrice']
        current_price_time = curr_data[str(id)]['prices']['ps']['updated']
        curr_price=curr_price.append({'Name': name, 'Current': current_price, 
                                      'Time': current_price_time}, ignore_index=True)

        hist_price['Date'] =  pd.to_datetime(hist_price['Date'], format='%Y-%m-%d')
        hist_price['Price'] = hist_price['Price'].astype('int64')
        curr_price['Current'] = curr_price['Current'].str.replace(',', '').astype('int64')

    names = list(hist_price['Name'].unique())

    for name in names:
        historical_price = hist_price[hist_price['Name']==name]['Price'].tail(30)
        current_price = curr_price[curr_price['Name']==name]['Current']

        plt.title(name)
        plt.axhline(y=current_price.iloc[0], color='r', linestyle='-')
        plt.plot(historical_price)
        plt.show() 

def get_tradepile_prices_old(profit_threshold=50):  #in percentage
    ## get prices of player cards in tradepile

    tradepile = core.tradepile()
    time.sleep(random.randint(1,3))

    tradepile_df = pd.DataFrame(columns=['itemId', 'resourceId', 'Name', 'Current', 'Time'])

    for card in tradepile:
        if card.get('tradeState')!='active' and card.get('itemType')=='player' and card.get('rareflag')!=0:

            item_id = card.get('id')
            #asset_id = card.get('assetId')
            resource_id = card.get('resourceId')
            last_sale_price = card.get('lastSalePrice')
            prp_max = card.get('marketDataMaxPrice')
            prp_min = card.get('marketDataMinPrice')

            #method which gets current prices from futbin
            current_price_df = get_current_price(resource_id)
            time.sleep(random.randint(7,10))

            #current_price_df = current_price_df.to_dict()

            temp_tradepile_df = pd.DataFrame(columns=['itemId', 'resourceId', 'Bought', 'Profit', 'PRP (Max)', 'PRP (Min)'])

            temp_tradepile_df = temp_tradepile_df.append({'itemId': item_id, 
                                                          'resourceId': resource_id, 
                                                          'Bought': last_sale_price,
                                                          'PRP (Max)': prp_max,
                                                          'PRP (Min)': prp_min
                                                        },
                                               ignore_index=True)


            #calculate profit
            temp_tradepile_df['Profit'] = current_price_df['Current'] - temp_tradepile_df['Bought']

            temp_tradepile_df['Profit %'] = temp_tradepile_df['Profit']/temp_tradepile_df['Bought'] * 100
            temp_tradepile_df['Sell?'] = int(np.where(temp_tradepile_df['Profit %'] >= profit_threshold, 1, 0))


            #concat the 2 dataframes
            temp_tradepile_df = pd.concat([temp_tradepile_df.drop(['resourceId'], axis='columns') , current_price_df], 
                                     axis=1, join='inner')

            #adds current row of data to actual tradepile_df
            tradepile_df = tradepile_df.append(temp_tradepile_df)

    print("TODO: don't get prices repeatedly for same player")
    print("TODO: fix all the extra zeros")
    return tradepile_df

def get_card_info(asset_id, resource_id):
    #gets info for cards and append info to a DataFrame

    tradepile = core.tradepile()

    for card in tradepile:
        #name = 

        trade_id = item['tradeId']
        buy_now_price = item['buyNowPrice']
        trade_state = item['tradeState']
        bid_state = item['bidState']
        starting_bid = i['startingBid']
        item_id = i['id']
        timestamp = i['timestamp']  # auction start
        rating = i['rating']
        asset_id = i['assetId']
        resource_id = i['resourceId']
        item_state = i['itemState']
        rareflag = i['rareflag']
        formation = i['formation']
        injury_type = i['injuryType']
        suspension = i['suspension']
        contract = i['contract']
        playStyle = i['playStyle']  # used only for players
        discardValue = i['discardValue']
        itemType = i['itemType']
        owners = i['owners']
        offers = i['offers']
        current_bid = i['currentBid']
        expires = i['expires']  # seconds left

def get_player_ids(min_rating, max_rating):
#take in a list or if no list, searches through database for players with rating x-y?

    players_db = fut.core.players()

    players_db_df = pd.DataFrame(columns=['assetId', 'resourceId', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY'])

    for key, value in players_db.items():

        #if key == 197083:
        asset_id = key

        name = players_db[asset_id].get('firstname') + ' ' + players_db[asset_id].get('lastname')

        all_versions = core.searchDefinition(asset_id) #returns a list

        for player in all_versions:
            resource_id = player['id']
            rating = player['rating']
            attribute_list = player['attributeList'] #returns another list

            #attributes list
            for index in range(len(attribute_list)): #this returns 0-5
                for attribute in attribute_list[index]:
                    if index == 0:
                        pace = attribute_list[index]['value']
                    elif index == 1:
                        shooting = attribute_list[index]['value']
                    elif index == 2:
                        passing = attribute_list[index]['value']
                    elif index == 3:
                        dribbling = attribute_list[index]['value']
                    elif index == 4:
                        defending = attribute_list[index]['value']
                    elif index == 5:
                        physical = attribute_list[index]['value']


            #append to dataframe
            players_db_df = players_db_df.append({'assetId': asset_id,
                                                 'resourceId': resource_id,
                                                  'Name': name, 
                                                  'OVR': rating,
                                                  'PAC': pace, 
                                                  'SHO': shooting, 
                                                  'PAS': passing, 
                                                  'DRI': dribbling, 
                                                  'DEF': defending, 
                                                  'PHY': physical
            }, ignore_index=True)

    #filter for ratings
    players_db_df = players_db_df[min_rating <= [players_db_df['OVR']] <= max_rating]

    #update dictionary       
    players_db_dict = players_db_df.set_index('resourceId').to_dict('index')

    #players_db_df.to_csv('player_info.csv')

    #players_db_df = pd.read_csv('player_info.csv')

    return players_db_df

def market_scan_old(min_rating, max_rating, max_budget_per_card=999999999):

    time_start = datetime.now() #testing

    shortlisted_df = futbin_html_scrape(min_rating = min_rating, 
                                        max_rating = max_rating, 
                                        max_price = max_budget_per_card) #returns a df

    shortlist_dict = shortlisted_df.set_index('resourceId').to_dict('index') #converts to dict to iterate

    #get info from EA official site
    ea_url = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr={0}:{1}&page=1'
    ea_json_data = requests.get(ea_url.format(min_rating, max_rating)).json()
    num_pages = ea_json_data['totalPages']
    #print(num_pages)

    num_results = ea_json_data['totalResults']

    players_db_df = pd.DataFrame(columns=['assetId', 'resourceId', 'Name', 
                                          'RAT', 'POS', 'VER', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 
                                          'Current', 'Time', 'Date Added',
                                        'YTD Mean', 'YTD Median', 'YTD Max', 'YTD Min', 
                                        '30d Mean', '30d Median', '30d Max', '30d Min',
                                       '14d Mean', '14d Median', '7d Mean', '7d Median',
                                       'Yest'
                                         ])

    for i in range(1): #num_pages

        url = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr={0}:{1}&page={2}'
            #gets info from EA official site
        players_db = requests.get(url.format(min_rating, max_rating, i+1)).json()['items'] #returns a list


        for player in players_db[9:13]: #player is dict; slicing to limit results/load
            name = player['name']
            attribute_list = player['attributes']

           #attributes list
            for index in range(len(attribute_list)): #this returns 0-5
                for attribute in attribute_list[index]:
                    if index == 0:
                        pace = attribute_list[index]['value']
                    elif index == 1:
                        shooting = attribute_list[index]['value']
                    elif index == 2:
                        passing = attribute_list[index]['value']
                    elif index == 3:
                        dribbling = attribute_list[index]['value']
                    elif index == 4:
                        defending = attribute_list[index]['value']
                    elif index == 5:
                        physical = attribute_list[index]['value']


            resource_id = player['id'] #100684097 for 99 c.ronaldo; this id for back end json prices; resource_id?
            asset_id = player['baseId'] #20801 for 99 c.ronaldo; asset_id?
            #futbin uses 17575 for 99 c. ronaldo front end

            rating = player['rating']
            position = player['position']
            version = player['rarityId']

            #https://fut.readthedocs.io/en/latest/user/lookuptables.html for rarityId values

            if version == 0:
                version = 'None'
            elif version == 1:
                version = 'Rare'
            elif version == 3:
                version = 'TOTW'
            elif version == 5:
                version = 'TOTY'
            elif version == 11:
                version = 'TOTS'
            elif version == 12:
                version = 'ICON'
            elif version == 15:
                version = 'HALLOWEEN' #don't search
            else:
                version == version

            print(name, rating, position, version, asset_id, resource_id) #testing

            #get current price data from FUTBIN
            current_price_url = 'https://www.futbin.com/19/playerPrices?player={0}'
            current_price_data = requests.get(current_price_url.format(resource_id)).json()
            #sleep
            time.sleep(random.randint(2,4))

            current_price = current_price_data[str(resource_id)]['prices']['ps']['LCPrice']

            if current_price == 0:
                continue
            else:
                current_price = int(current_price.replace(',', ''))

            #current_price2 = current_price_data[str(resource_id)]['prices']['ps']['LCPrice2']
                #up till LCPrice5; might wanna add

            current_price_time = current_price_data[str(resource_id)]['prices']['ps']['updated']
            #might want to review this timing, take actual time so that this will be accurate


            if 0 < current_price < max_budget_per_card:

                 #hist price data
                hist_price_df = pd.DataFrame(columns=['Date', 'Price'])

                hist_price_url = 'https://www.futbin.com/19/playerGraph?type=daily_graph&year=19&player={0}'
                hist_price_data = requests.get(hist_price_url.format(resource_id)).json()

                #sleep
                time.sleep(random.randint(2,4))

                for price in hist_price_data['ps']:
                    #There is extra zeroes in response.
                    date = datetime.utcfromtimestamp(price[0] / 1000).strftime('%Y-%m-%d')
                    price = price[1]

                    hist_price_df = hist_price_df.append({'Date': date, 
                                                    'Price': price}, ignore_index=True)

                mean_ytd = int(round(hist_price_df['Price'].mean()))
                median_ytd = int(round(hist_price_df['Price'].median()))
                max_ytd = hist_price_df['Price'].max()
                min_ytd = hist_price_df['Price'].min()

                mean_30d = int(round(hist_price_df['Price'].tail(30).mean()))
                median_30d = int(round(hist_price_df['Price'].tail(30).median()))
                max_30d = hist_price_df['Price'].tail(30).max()
                min_30d = hist_price_df['Price'].tail(30).min()

                mean_14d = int(round(hist_price_df['Price'].tail(14).mean()))
                median_14d = int(round(hist_price_df['Price'].tail(14).median()))

                mean_7d = int(round(hist_price_df['Price'].tail(7).mean()))
                median_7d = int(round(hist_price_df['Price'].tail(7).median()))

                price_yest = hist_price_df['Price'].tail(1).iloc[0]

                date_added = hist_price_df['Date'].head(1).iloc[0]

                #append to dataframe
                players_db_df = players_db_df.append({'assetId': asset_id,
                                                    'resourceId': resource_id,
                                                    'Name': name,
                                                    'RAT': rating,
                                                    'POS': position,
                                                    'VER': version,
                                                    'PAC': pace, 
                                                    'SHO': shooting, 
                                                    'PAS': passing, 
                                                    'DRI': dribbling, 
                                                    'DEF': defending, 
                                                    'PHY': physical,
                                                    'Current': current_price,
                                                    'Time': current_price_time,
                                                      'Date Added': date_added,
                                                      'YTD Mean': mean_ytd, 
                                                      'YTD Median': median_ytd, 
                                                      'YTD Max': max_ytd, 
                                                      'YTD Min': min_ytd, 
                                                      '30d Mean': mean_30d, 
                                                      '30d Median': median_30d, 
                                                      '30d Max': max_30d, 
                                                      '30d Min': min_30d,
                                                      '14d Mean': mean_14d, 
                                                      '14d Median': median_14d, 
                                                      '7d Mean': mean_7d, 
                                                      '7d Median': median_7d,
                                                       'Yest': price_yest
                                                     }, ignore_index=True)   
            else:
                continue
                print('FAIL') 
                #print(name, rating, position, version, asset_id, resource_id + " is not added")

            time.sleep(random.randint(10,15))
    #
    time_end = datetime.now()
    time_delta = time_end - time_start

    print(players_db_df.shape[0] + " rows in " + time_delta + " seconds")

    return players_db_df

def get_player_ids_ea(max_rating, min_rating=75):    
    #get info from EA official site
    ea_url = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr={0}:{1}&page=1'
    ea_json_data = requests.get(ea_url.format(min_rating, max_rating)).json()
    num_pages = ea_json_data['totalPages']
    print(num_pages)

    num_results = ea_json_data['totalResults']

    players_db_df = pd.DataFrame(columns=['assetId', 'resourceId', 'Name', 
                                          'RAT', 'POS', 'VER', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY'])

    for i in range(num_pages): #num_pages

        url = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr={0}:{1}&page={2}'
            #gets info from EA official site
            #sleep
        time.sleep(random.randint(5,10))
        players_db = requests.get(url.format(min_rating, max_rating, i+1)).json()['items'] 
            #returns a list; i+1 to start from page1

        for player in players_db: #player is dict; slicing to limit results/load
            name = player['name']
            attribute_list = player['attributes']

           #attributes list
            for index in range(len(attribute_list)): #this returns 0-5
                for attribute in attribute_list[index]:
                    if index == 0:
                        pace = attribute_list[index]['value']
                    elif index == 1:
                        shooting = attribute_list[index]['value']
                    elif index == 2:
                        passing = attribute_list[index]['value']
                    elif index == 3:
                        dribbling = attribute_list[index]['value']
                    elif index == 4:
                        defending = attribute_list[index]['value']
                    elif index == 5:
                        physical = attribute_list[index]['value']


            resource_id = player['id'] #100684097 for 99 c.ronaldo; this id for back end json prices; resource_id?
            asset_id = player['baseId'] #20801 for 99 c.ronaldo; asset_id?
            #futbin uses 17575 for 99 c. ronaldo front end

            rating = player['rating']
            position = player['position']
            version = player['rarityId']

            #https://fut.readthedocs.io/en/latest/user/lookuptables.html for rarityId values

            if version == 0:
                version = 'None'
            elif version == 1:
                version = 'Rare'
            elif version == 3:
                version = 'TOTW'
            elif version == 5:
                version = 'TOTY'
            elif version == 11:
                version = 'TOTS'
            elif version == 12:
                version = 'ICON'
            elif version == 15:
                version = 'HALLOWEEN' #don't search
            else:
                version == version

            #print(name, rating, position, version, asset_id, resource_id) #testing

            players_db_df = players_db_df.append({'assetId': asset_id,
                                                    'resourceId': resource_id,
                                                    'Name': name,
                                                    'RAT': rating,
                                                    'POS': position,
                                                    'VER': version,
                                                    'PAC': pace, 
                                                    'SHO': shooting, 
                                                    'PAS': passing, 
                                                    'DRI': dribbling, 
                                                    'DEF': defending, 
                                                    'PHY': physical}, ignore_index=True)

    return players_db_df


def futbin_html_scrape_old(min_rating=75, max_rating=99, min_price=200, max_price=99999999999, version='gold_rare'):

    if version != None:
        version = '&version=' + version
    if min_rating != None:
        min_rating = '&player_rating=' + str(min_rating) #this line is useless atm

    first_page_url = 'https://www.futbin.com/19/players?page=1&player_rating={0}-{1}&ps_price={2}-{3}{4}&sort=ps_price&order=desc'

    first_page_url_format = first_page_url.format(min_rating, max_rating, min_price, max_price, version)
        #'https://www.futbin.com/19/players?page={}&league=31&ps_price=200-20000'.format(i)
    first_page = requests.get(first_page_url_format)
        #sleep
    time.sleep(random.randint(2,4))

    A=[] #resource_id
    B=[] #name
    C=[] #current price

    first_page_soup = BeautifulSoup(first_page.text, 'lxml')

    if first_page_soup.find_all('li', class_='page-item ') == []:
        num_pages = 1
    else:
        num_pages = int(first_page_soup.find_all('li', class_='page-item ')[-1].find('a').find(text=True))

    print(str(num_pages) + " pages/requests: " + str(num_pages * 30) + " players expected")

    print("Expected time to complete: " + str(num_pages*30/6) + "mins")

    for i in range(num_pages):
        url = 'https://www.futbin.com/19/players?page={0}&player_rating={1}-{2}&ps_price={3}-{4}{5}&sort=ps_price&order=desc'
        url_format = url.format(i+1, min_rating, max_rating, min_price, max_price, version) #i+1 to start from page 1 rather than 0
        page = requests.get(url_format)
            #sleep
        time.sleep(random.randint(2,4))
        soup = BeautifulSoup(page.text, 'lxml')

        #code for individual page
        table = soup.find('table', class_='table table-bordered table-hover table-responsive w-100 d-block d-md-table')

        for row in table.find_all('tr'):

            player_bio = row.find('td', class_='table-row-text row') 

            #find resource_id; this is wrong
            resource_id_container = player_bio.find('div', class_='d-inline')
            resource_id_link = resource_id_container.find_all('img')[0].get('data-original')

            resource_id_index1 = resource_id_link.find('players/')
            resource_id_index2 = resource_id_link.find('.png')

            resource_id = resource_id_link[resource_id_index1 + 8: resource_id_index2] #plus 8 cos 'players/' is 8 chars

            resource_id = resource_id.replace('p', '')

            #find player name
            player_name_container = player_bio.find('div', class_='d-inline pt-2 pl-3')

            name = player_name_container.find_all('div')[0].find('a', class_='player_name_players_table').find(text=True)

            #get current price
            current_price = str(row.find_all('td')[4].find(text=True)) #convert to string

            if current_price.find('K')!=-1:
                current_price = float(current_price.replace('K', '')) * 1000 #change to int?
            elif current_price.find('M')!=-1:
                current_price = float(current_price.replace('M', '')) * 1000000
            else:
                current_price = float(current_price)

            current_price = int(current_price)

            ############## might wanna add a method to get stats here ########################

            ############## might wanna add a method to write to csv here ########################


            A.append(resource_id)
            B.append(name)
            C.append(current_price)

        df = pd.DataFrame()
        df['resourceId'] = A
        df['Name'] = B
        df['Current'] = C
        #end of futbin html data

    return df

def get_num_searches_ea(min_rating, max_rating):

    url = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr={0}:{1}&page=1'
    json_data = requests.get(url.format(min_rating, max_rating)).json()
    num_pages = json_data['totalPages']
    num_results = json_data['totalResults']

    print(str(num_pages) + " pages")
    print(num_results)

    return None

def get_current_price_old(resource_id): 
    #gets current price of player from futbin and appends data to DataFrame

    
    name = get_player_name(resource_id)

    #current price data
    url = requests.get('https://www.futbin.com/19/playerPrices?player={0}'.format(resource_id))
    current_price_data = url.json()
        #sleep
    time.sleep(random.randint(1,3))
    current_price = int(current_price_data[str(resource_id)]['prices']['ps']['LCPrice'].replace(',', ''))
    current_price_time = current_price_data[str(resource_id)]['prices']['ps']['updated']

    #creates DataFrame to store info
    current_price_df = pd.DataFrame(columns=['resourceId', 'Name', 'Current', 'Time'])
    current_price_df=current_price_df.append({#'assetId': asset_id,
                                              'resourceId': resource_id, 
                                              'Name': name, 
                                              'Current': current_price, 
                                             'Time': current_price_time}, ignore_index=True)

    '''
    current_price_url = 'https://www.futbin.com/19/playerPrices?player={0}'
    current_price_data = requests.get(current_price_url.format(resource_id)).json()
    sleep
    time.sleep(random.randint(2,4))

    current_price = current_price_data[str(resource_id)]['prices']['ps']['LCPrice']

    if current_price == 0:
        continue
    else:
        current_price = int(current_price.replace(',', ''))

    current_price2 = current_price_data[str(resource_id)]['prices']['ps']['LCPrice2']
        # up till LCPrice5; might wanna add

    current_price_time = current_price_data[str(resource_id)]['prices']['ps']['updated']
        # might want to review this timing, take actual time so that this will be accurate

    if 0 < current_price < max_price:
    '''    
    print("TODO: change this to take in a dict/df instead of just resource_id, similar to get hist_price")
    
    return current_price_df